In [2]:
# Dependencies
import pandas as pd
import numpy as np

In [3]:
# Import files using pandas
items_df = pd.read_csv("generated_data\items_complete.csv")
players_df = pd.read_csv("generated_data\players_complete.csv")
purchase_df = pd.read_csv("generated_data\purchase_data_3.csv")

In [4]:
# Get the total number of players with .count(), using SN column b/c its short
player_count = players_df["SN"].count()

In [5]:
# Get the total count of unique items with len() since .unique() returns a list
unique_item_count = len(items_df["Item Name"].unique())

In [6]:
# Get the average purchase price by dividing the sum of the price column by total number of items
average_purchase_price = purchase_df["Price"].sum() / len(purchase_df["Price"])

In [7]:
# Get the total number of purchases
total_purchases = purchase_df["Price"].count()

In [8]:
# Get total revenue by taking price from purchase and subtracting price from item
# join table -> do more stuff
#purchase_items_joined_df = pd.merge(purchase_df, items_df, on="Item ID")
#purchase_items_joined_df = purchase_items_joined_df.rename(columns={"Item Name_x": "Item Name(Purchase)",
#                                                                   "Price_x": "Purchase Price",
#                                                                   "Item Name_y": "Item Name(Stock)",
#                                                                   "Price_y": "MSRP"})
#purchase_items_joined_df["Revenue"] = purchase_items_joined_df["Purchase Price"] - purchase_items_joined_df["MSRP"]
#purchase_items_joined_df.head() FFFFFFFF I SHOULD'VE CHECKED THE GENERATOR CODE TO SEE IF THE PRICE WAS DIFFERENT
# OK actually just add up the revenue from the purchase_df
total_revenue = purchase_df["Price"].sum()

In [9]:
# Get % and # of male players
number_male = len(players_df.loc[players_df["Gender"] == "Male",:])

percent_male = (number_male / player_count)
percent_male = "{:.2%}".format(percent_male)
percent_male

'82.03%'

In [10]:
# Get % and # of female players
number_female = len(players_df.loc[players_df["Gender"] == "Female",:])

percent_female = (number_female / player_count)
percent_female = "{:.2%}".format(percent_female)
percent_female

'16.08%'

In [11]:
# Get % and # of other / non-disclosed players
number_other = len(players_df.loc[players_df["Gender"] == "Other / Non-Disclosed",:])

percent_other = (number_other / player_count)
percent_other = "{:.2%}".format(percent_other)
percent_other

'1.89%'

In [12]:
# Set up some dataframes etc to use for purchase analysis
male_df = purchase_df.loc[purchase_df["Gender"] == "Male",:]
female_df = purchase_df.loc[purchase_df["Gender"] == "Female",:]
other_df = purchase_df.loc[purchase_df["Gender"] == "Other / Non-Disclosed",:]

In [13]:
# Get purchase count broken by gender
male_purchase = len(male_df)
female_purchase = len(female_df)
other_purchase = len(other_df)

In [14]:
# Get total purchase value broken by gender
male_total_value = male_df["Price"].sum()
female_total_value = female_df["Price"].sum()
other_total_value = other_df["Price"].sum()

In [15]:
# Get average purchase price broken by gender
male_average_price = male_total_value / male_purchase

female_average_price = female_total_value / female_purchase

other_average_price = other_total_value / other_purchase

In [16]:
# Get normalized totals broken by gender, with code repurposed from stackoverflow
male_purchase_normalized = (male_df["Price"] - male_df["Price"].mean()) / (male_df["Price"].max() - male_df["Price"].min())
male_purchase_normalized_total = male_purchase_normalized.sum()

female_purchase_normalized = (female_df["Price"] - female_df["Price"].mean()) / (female_df["Price"].max() - female_df["Price"].min())
female_purchase_normalized_total = female_purchase_normalized.sum()

other_purchase_normalized = (other_df["Price"] - other_df["Price"].mean()) / (other_df["Price"].max() - other_df["Price"].min())
other_purchase_normalized_total = other_purchase_normalized.sum()

In [24]:
# Get purchase count by age in bins of 4 years
# first join purchase and players tables and clean it
players_purchase_joined_df = pd.merge(purchase_df, players_df, on="SN")
players_purchase_joined_df = players_purchase_joined_df.rename(columns={"Age_x": "Age",
                                                                       "Gender_x": "Gender"})
players_purchase_joined_clean_df = players_purchase_joined_df[["Purchase ID",
                                                             "SN",
                                                             "Age",
                                                             "Gender",
                                                             "Item ID",
                                                             "Item Name",
                                                             "Price",
                                                             "Player ID"]]

In [91]:
purchase_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Iloni35,20,Male,93,Apocalyptic Battlescythe,4.49
1,1,Aidaira26,21,Male,12,Dawne,3.36
2,2,Irim47,17,Male,5,Putrid Fan,2.63
3,3,Irith83,17,Male,123,Twilight's Carver,2.55
4,4,Philodil43,22,Male,154,Feral Katana,4.11


In [44]:
items_df.head(1)

,Item ID,Item Name,Price
0,0,Splinter,1.89


In [79]:
players_df.head(10)

,Player ID,SN,Age,Gender
0,0,Marughi89,21,Male
1,1,Lirtedy26,40,Male
2,2,Chamistast30,7,Male
3,3,Lisirra25,24,Male
4,4,Lirtim36,23,Male
5,5,Undimsya85,17,Female
6,6,Lirtassa77,20,Female
7,7,Mindirra92,23,Male
8,8,Undirrasta89,23,Male
9,9,Iskjaskst81,15,Male


In [ ]:
# Things to do
#PLAYER COUNT
# - Total Number of Players
#PURCHASING ANALYSIS(TOTAL)
# - Number of Unique Items
# - Average Purchase Price
# - Total Number of Purchases
# - Total Revenue
#GENDER DEMOGRAPHICS
# - Percentage and Count of Male Players
# - Percentage and Count of Female Players
# - Percentage and Count of Other / Non-Disclosed
#PURCHASING ANALYSIS (GENDER)
#>broken by gender
# - Purchase Count
# - Average Purchase Price
# - Total Purchase Value
# - Normalized Totals
#AGE DEMOGRAPHICS
#>broken into bins of 4 years (ie <10, 10-14, 15-19,...)
# - Purchase Count
#TOP SPENDERS
#MOST POPULAR ITEMS
#MOST PROFITABLE ITEMS